In [ ]:
!pip install ctgan
!pip install sdv

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from ctgan import CTGAN

import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import time

start=time.time()

dataset = pd.read_csv('https://raw.githubusercontent.com/mahayasa/various-sampling-churn-prediction/main/data/mobile-churn.csv')


df1=dataset[dataset['churn']==0]
df=dataset[dataset['churn']==1]

#filling missing value with mean
df = df.fillna(df.mean())
df=df.drop(['year','month','user_account_id'],axis=1)

#filling missing value with mean
df1 = df1.fillna(df1.mean())
df1=df1.drop(['year','month','user_account_id'],axis=1)

batch_size = 5000
epochs = 100
model = CTGAN(batch_size=batch_size, epochs=epochs, verbose=True)
model.fit(df)

end=time.time()
print("The time of execution of preprocess:",
      (end-start), "s")




Gen. (0.00) | Discrim. (0.00):   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at ../aten/src/ATen/cuda/CublasHandlePool.cpp:135.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Gen. (-6.06) | Discrim. (-0.14): 100%|██████████| 100/100 [00:25<00:00,  3.88it/s]

The time of execution of preprocess: 100.22761106491089 s


In [ ]:
n_generated_data = 38655
generated_df = model.sample(n_generated_data)

#concat original data and gan data
data_concat = pd.concat([df, generated_df])
# combine data churn and not churn
data=pd.concat([df1, data_concat])


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun  1 17:31:40 2020

@author: manav

Modifed on 23 AUG 2022

by mahayasa adiputra
"""

import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score
import sklearn.metrics as mt
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from numpy import mean
from numpy import std
from sklearn.metrics import make_scorer
from imblearn.metrics import specificity_score
import math
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#data=data.dropna()
#data=data.fillna(0)

start1=time.time()



X=data.drop(['churn'],axis=1)
y=data["churn"]

#enn = EditedNearestNeighbours(n_neighbors=3)
#X, y = enn.fit_resample(X, y)
#ncr = NeighbourhoodCleaningRule(n_neighbors=5, kind_sel='all')
#X, y = ncr.fit_resample(X, y)
#tomek_links = TomekLinks()
#X, y = tomek_links.fit_resample(X, y)


#kfold cross validation
cv = KFold(n_splits=5, random_state=1, shuffle=True)

end1=time.time()
print("The time of execution of preprocess:",
      (end1-start1), "s")



The time of execution of preprocess: 0.04720568656921387 s


In [ ]:
# decision tree
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-mobile-dt-cs-auc.xlsx', index=False)

Best: 0.909602 using {'class_weight': {0: 100, 1: 90}}
0.906916 (0.002278) with: {'class_weight': {0: 100, 1: 10}}
0.907116 (0.001642) with: {'class_weight': {0: 100, 1: 20}}
0.907298 (0.001478) with: {'class_weight': {0: 100, 1: 30}}
0.907210 (0.001792) with: {'class_weight': {0: 100, 1: 40}}
0.907603 (0.002249) with: {'class_weight': {0: 100, 1: 50}}
0.908474 (0.002022) with: {'class_weight': {0: 100, 1: 60}}
0.909115 (0.002375) with: {'class_weight': {0: 100, 1: 70}}
0.909285 (0.001882) with: {'class_weight': {0: 100, 1: 80}}
0.909602 (0.001680) with: {'class_weight': {0: 100, 1: 90}}
0.909312 (0.001650) with: {'class_weight': {0: 100, 1: 100}}


In [7]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-dt-cs-f1.xlsx', index=False)

Best: 0.892929 using {'class_weight': {0: 100, 1: 90}}
0.858683 (0.002194) with: {'class_weight': {0: 100, 1: 10}}
0.859722 (0.001908) with: {'class_weight': {0: 100, 1: 20}}
0.884100 (0.001711) with: {'class_weight': {0: 100, 1: 30}}
0.884588 (0.001825) with: {'class_weight': {0: 100, 1: 40}}
0.885724 (0.001787) with: {'class_weight': {0: 100, 1: 50}}
0.886910 (0.001996) with: {'class_weight': {0: 100, 1: 60}}
0.891467 (0.001554) with: {'class_weight': {0: 100, 1: 70}}
0.892565 (0.001484) with: {'class_weight': {0: 100, 1: 80}}
0.892929 (0.001506) with: {'class_weight': {0: 100, 1: 90}}
0.892610 (0.001349) with: {'class_weight': {0: 100, 1: 100}}


In [8]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-dt-cs-gmean.xlsx', index=False)

Best: 0.892496 using {'class_weight': {0: 100, 1: 90}}
0.857963 (0.002246) with: {'class_weight': {0: 100, 1: 10}}
0.858284 (0.001636) with: {'class_weight': {0: 100, 1: 20}}
0.883970 (0.001504) with: {'class_weight': {0: 100, 1: 30}}
0.884329 (0.001936) with: {'class_weight': {0: 100, 1: 40}}
0.885582 (0.001624) with: {'class_weight': {0: 100, 1: 50}}
0.886906 (0.001960) with: {'class_weight': {0: 100, 1: 60}}
0.891466 (0.001542) with: {'class_weight': {0: 100, 1: 70}}
0.892253 (0.001612) with: {'class_weight': {0: 100, 1: 80}}
0.892496 (0.001787) with: {'class_weight': {0: 100, 1: 90}}
0.892273 (0.001468) with: {'class_weight': {0: 100, 1: 100}}


In [9]:
# LR
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-lr-cs-auc.xlsx', index=False)

Best: 0.928182 using {'class_weight': {0: 100, 1: 90}}
0.904492 (0.002064) with: {'class_weight': {0: 100, 1: 10}}
0.918634 (0.001758) with: {'class_weight': {0: 100, 1: 20}}
0.923747 (0.002014) with: {'class_weight': {0: 100, 1: 30}}
0.925480 (0.001828) with: {'class_weight': {0: 100, 1: 40}}
0.927666 (0.001507) with: {'class_weight': {0: 100, 1: 50}}
0.927268 (0.001457) with: {'class_weight': {0: 100, 1: 60}}
0.927132 (0.001773) with: {'class_weight': {0: 100, 1: 70}}
0.927611 (0.002022) with: {'class_weight': {0: 100, 1: 80}}
0.928182 (0.001920) with: {'class_weight': {0: 100, 1: 90}}
0.928049 (0.001729) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
from sklearn.metrics import f1_score, make_scorer
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-lr-cs-f1.xlsx', index=False)

Best: 0.884072 using {'class_weight': {0: 100, 1: 90}}
0.684735 (0.006808) with: {'class_weight': {0: 100, 1: 10}}
0.767990 (0.004258) with: {'class_weight': {0: 100, 1: 20}}
0.814115 (0.002032) with: {'class_weight': {0: 100, 1: 30}}
0.844235 (0.003191) with: {'class_weight': {0: 100, 1: 40}}
0.862411 (0.001797) with: {'class_weight': {0: 100, 1: 50}}
0.874548 (0.002600) with: {'class_weight': {0: 100, 1: 60}}
0.880589 (0.002508) with: {'class_weight': {0: 100, 1: 70}}
0.882865 (0.003026) with: {'class_weight': {0: 100, 1: 80}}
0.884072 (0.002361) with: {'class_weight': {0: 100, 1: 90}}
0.882048 (0.003210) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from imblearn.metrics import geometric_mean_score
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-lr-cs-gmean.xlsx', index=False)

Best: 0.883918 using {'class_weight': {0: 100, 1: 90}}
0.654514 (0.008027) with: {'class_weight': {0: 100, 1: 10}}
0.754748 (0.004903) with: {'class_weight': {0: 100, 1: 20}}
0.806552 (0.002357) with: {'class_weight': {0: 100, 1: 30}}
0.840967 (0.003409) with: {'class_weight': {0: 100, 1: 40}}
0.860578 (0.001842) with: {'class_weight': {0: 100, 1: 50}}
0.873942 (0.002683) with: {'class_weight': {0: 100, 1: 60}}
0.880399 (0.002536) with: {'class_weight': {0: 100, 1: 70}}
0.882829 (0.003026) with: {'class_weight': {0: 100, 1: 80}}
0.883918 (0.002370) with: {'class_weight': {0: 100, 1: 90}}
0.881653 (0.003225) with: {'class_weight': {0: 100, 1: 100}}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#SVM
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
# define grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-svm-cs-auc.xlsx', index=False)

Best: 0.964091 using {'class_weight': {0: 100, 1: 100}}
0.947882 (0.001695) with: {'class_weight': {0: 100, 1: 10}}
0.955491 (0.001373) with: {'class_weight': {0: 100, 1: 20}}
0.956769 (0.001332) with: {'class_weight': {0: 100, 1: 30}}
0.957529 (0.001288) with: {'class_weight': {0: 100, 1: 40}}
0.958600 (0.001324) with: {'class_weight': {0: 100, 1: 50}}
0.960189 (0.001385) with: {'class_weight': {0: 100, 1: 60}}
0.962289 (0.001545) with: {'class_weight': {0: 100, 1: 70}}
0.962569 (0.001538) with: {'class_weight': {0: 100, 1: 80}}
0.963146 (0.001527) with: {'class_weight': {0: 100, 1: 90}}
0.964091 (0.001497) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
f1 = make_scorer(f1_score , average='macro')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=f1)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-svm-cs-f1.xlsx', index=False)

Best: 0.910680 using {'class_weight': {0: 100, 1: 100}}
0.847716 (0.001809) with: {'class_weight': {0: 100, 1: 10}}
0.894612 (0.001469) with: {'class_weight': {0: 100, 1: 20}}
0.895527 (0.001506) with: {'class_weight': {0: 100, 1: 30}}
0.898961 (0.001432) with: {'class_weight': {0: 100, 1: 40}}
0.899343 (0.001538) with: {'class_weight': {0: 100, 1: 50}}
0.903892 (0.002044) with: {'class_weight': {0: 100, 1: 60}}
0.904438 (0.001389) with: {'class_weight': {0: 100, 1: 70}}
0.910642 (0.001845) with: {'class_weight': {0: 100, 1: 80}}
0.910668 (0.001859) with: {'class_weight': {0: 100, 1: 90}}
0.910680 (0.001867) with: {'class_weight': {0: 100, 1: 100}}


In [ ]:
from imblearn.metrics import geometric_mean_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import f1_score, make_scorer
model = SVC()
# define grid
balance1 = [{0:10,1:100},{0:20,1:100},{0:30,1:100},{0:40,1:100},{0:50,1:100},{0:60,1:100},{0:70,1:100},{0:80,1:100},{0:90,1:100},{0:100,1:100}]
balance = [{0:100,1:10},{0:100,1:20},{0:100,1:30},{0:100,1:40},{0:100,1:50},{0:100,1:60},{0:100,1:70},{0:100,1:80},{0:100,1:90},{0:100,1:100}]
param_grid = dict(class_weight=balance)
gm_scorer = make_scorer(geometric_mean_score, greater_is_better=True, average='binary')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring=gm_scorer)
# execute the grid search
grid_result = grid.fit(X, y)
# report the best configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# report all configurations
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Create a DataFrame with the results
results_df = pd.DataFrame(grid_result.cv_results_)
# Save the DataFrame to an Excel file
results_df.to_excel('ctgan-tn-mobile-svm-cs-gmean.xlsx', index=False)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best: 0.910829 using {'class_weight': {0: 100, 1: 100}}
0.842336 (0.002115) with: {'class_weight': {0: 100, 1: 10}}
0.884379 (0.017549) with: {'class_weight': {0: 100, 1: 20}}
0.894820 (0.002154) with: {'class_weight': {0: 100, 1: 30}}
0.898619 (0.002125) with: {'class_weight': {0: 100, 1: 40}}
0.898753 (0.002200) with: {'class_weight': {0: 100, 1: 50}}
0.898856 (0.002230) with: {'class_weight': {0: 100, 1: 60}}
0.903966 (0.002194) with: {'class_weight': {0: 100, 1: 70}}
0.910430 (0.001923) with: {'class_weight': {0: 100, 1: 80}}
0.910752 (0.001054) with: {'class_weight': {0: 100, 1: 90}}
0.910829 (0.001046) with: {'class_weight': {0: 100, 1: 100}}
